# go-web表单

### 处理表单输入

默认情况下，Handler里面是不会自动解析form的，必须显式的调用r.ParseForm()后，你才能对这个表单数据进行操作。

r.Form里面包含了所有请求参数，比如URL中query-string、POST的数据、PUT的数据，当你在URL中的query-string字段和POST冲突时，会保存成一个slice，里面存储了多个值，Go官方文档中说在接下来的版本里面将会把POST、GET这些数据分离开来。

request.Form是一个url.Values类型，里面存储的是对应的类似key=value的信息

Request本身也提供了FormValue()函数来获取用户提交的参数。如`r.Form["username"]`也可写成`r.FormValue("username")`。调用r.FormValue时会自动调用r.ParseForm，所以不必提前调用。r.FormValue只会返回同名参数中的第一个，若参数不存在则返回空字符串。

### 验证表单输入

开发Web的一个原则就是，不能信任用户输入的任何信息，所以验证和过滤用户的输入信息就变得非常重要.

一般做验证是前台做简单的验证,后台也要做验证. 前台是暴露给客户的,所以一般前台的验证大多可以被绕过,因此真正做验证的应该在后台.后台做的验证才是最有效的.

### 预防XSS

动态站点会受到XSS的威胁，而静态站点则完全不受其影响。

攻击者通常会在有漏洞的程序中插入JavaScript、VBScript、 ActiveX或Flash以欺骗用户。一旦得手，他们可以盗取用户帐户信息，修改用户设置，盗取/污染cookie和植入恶意广告等。

对XSS最佳的防护应该结合以下两种方法：
1. 验证所有输入数据，有效检测攻击
2. 对所有输出数据进行适当的处理，以防止任何已成功注入的脚本在浏览器端运行

### 防止多次提交表单

由于种种原因，用户经常会重复递交表单。通常这只是鼠标的误操作:
* 双击了递交按钮
* 也可能是为了编辑或者再次核对填写过的信息，点击了浏览器的后退按钮，然后又再次点击了递交按钮而不是浏览器的前进按钮。
* 也可能是故意的

解决方案: 在表单中添加一个带有唯一值的隐藏字段。在验证表单时，先检查带有该唯一值的表单是否已经递交过了。如果是，拒绝再次递交；如果不是，则处理表单进行逻辑处理。另外，如果是采用了Ajax模式递交表单的话，当表单递交后，通过javascript来禁用表单的递交按钮。

### 处理文件上传

用户上传文件有很多种情况，如存储用户拍摄的照片等等

要使表单能够上传文件，首先第一步就是要添加form的enctype属性，enctype属性有如下三种情况:
* application/x-www-form-urlencoded   表示在发送前编码所有字符（默认）
* multipart/form-data	  不对字符编码。在使用包含文件上传控件的表单时，必须使用该值。
* text/plain	  空格转换为 "+" 加号，但不对特殊字符编码。

上传文件主要三步处理：

1. 表单中增加enctype="multipart/form-data"
2. 服务端调用r.ParseMultipartForm,把上传的文件存储在内存和临时文件中
3. 使用r.FormFile获取文件句柄，然后对文件进行存储等处理。